In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import numpy as np

# 定义CNN模型
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 7*7*64)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 实例化模型
model = CNNNet()

# 加载MNIST数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 训练模型
for epoch in range(10):  # 假设我们训练10个epoch
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        '''
        # 新增代码：保存图像为BMP文件
        if batch_idx == 0 and epoch == 0:  # 仅在第一个epoch的第一个batch时保存图像，避免大量保存
            for i in range(min(len(data), 10)):  # 保存前10张图像为例
                img = data[i][0]  # 因为MNIST是灰度图像，所以取第一个通道
                img = img.detach().numpy()  # 转换为numpy数组
                img = img.squeeze()  # 移除可能的单维度条目
                img = (img - img.min()) / (img.max() - img.min()) * 255  # 归一化到0-255
                img = img.astype(np.uint8)  # 转为无符号整型
                filename = f'mnist_image_epoch{epoch}_batch{batch_idx}_idx{i}.bmp'
                cv2.imwrite(filename, img)  # 使用OpenCV保存图像，确保已安装cv2库
                print(f'Saved image to {filename}')
        '''
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            #print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\t')

# 保存模型
torch.save(model.state_dict(), 'mnist_cnn.pth')
print('Train finished!')

Saved image to mnist_image_epoch0_batch0_idx0.bmp
Saved image to mnist_image_epoch0_batch0_idx1.bmp
Saved image to mnist_image_epoch0_batch0_idx2.bmp
Saved image to mnist_image_epoch0_batch0_idx3.bmp
Saved image to mnist_image_epoch0_batch0_idx4.bmp
Saved image to mnist_image_epoch0_batch0_idx5.bmp
Saved image to mnist_image_epoch0_batch0_idx6.bmp
Saved image to mnist_image_epoch0_batch0_idx7.bmp
Saved image to mnist_image_epoch0_batch0_idx8.bmp
Saved image to mnist_image_epoch0_batch0_idx9.bmp
Train Epoch: 0 [0/60000 (0%)]	
Train Epoch: 0 [6400/60000 (11%)]	
Train Epoch: 0 [12800/60000 (21%)]	
Train Epoch: 0 [19200/60000 (32%)]	
Train Epoch: 0 [25600/60000 (43%)]	
Train Epoch: 0 [32000/60000 (53%)]	
Train Epoch: 0 [38400/60000 (64%)]	
Train Epoch: 0 [44800/60000 (75%)]	
Train Epoch: 0 [51200/60000 (85%)]	
Train Epoch: 0 [57600/60000 (96%)]	
Train Epoch: 1 [0/60000 (0%)]	
Train Epoch: 1 [6400/60000 (11%)]	
Train Epoch: 1 [12800/60000 (21%)]	
Train Epoch: 1 [19200/60000 (32%)]	
Train Ep

KeyboardInterrupt: 